In [1]:
%%capture
!pip install numpy matplotlib scikit-image plotly tensorflow torch pandas xgboost scikit-learn

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from os import listdir
import plotly.express as px
import skimage
from skimage.measure import block_reduce
import pandas as pd
from xgboost import XGBRegressor
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from pathlib import Path
from google.colab import drive

## Load and extract data

## Get Private Test Image & Load

In [ ]:
#This notebook here is the cleaned-up version with the sole goal of loading & predicting the (private) test image.
#The training of the model that is loaded here is done in the training notebook.

In [ ]:
#Get Private Test Image - replace path if other image to be predicted
!wget -O private_test_image_reduced.npy "https://uni-muenster.sciebo.de/s/Ye3GpOyPj0rCeTh/download?path=%2F&files=private_test_image_reduced.npy"

--2023-07-04 13:46:06--  https://uni-muenster.sciebo.de/s/Ye3GpOyPj0rCeTh/download?path=%2F&files=private_test_image_reduced.npy
Resolving uni-muenster.sciebo.de (uni-muenster.sciebo.de)... 128.176.1.2
Connecting to uni-muenster.sciebo.de (uni-muenster.sciebo.de)|128.176.1.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 335544448 (320M) [application/octet-stream]
Saving to: ‘private_test_image_reduced.npy’

private_test_image_ 100%[===================>] 320.00M  19.1MB/s    in 18s     

2023-07-04 13:46:25 (17.9 MB/s) - ‘private_test_image_reduced.npy’ saved [335544448/335544448]



In [ ]:
#Load Image in NP Arrays, transpose axis correctly
private_test = np.array([np.load(Path(f'private_test_image_reduced.npy'))]).transpose(2,3,1,0))

In [ ]:
#Reshape test image to remove additional dimension
X, Y, C, N = private_test.shape
private_test = private_test.reshape(X,Y,-1)

## Create Cropped Images & Save Them in DF

In [ ]:
#Produce cropped images as seen in image preprocessing for Model Training - adjust to include all pixels instead of just keypoints

CROPPING_OFFSET = 2 # Defines the pixel offset in every direction

# Padding images to allow cropping of edge values
padding_tuple = ((CROPPING_OFFSET,CROPPING_OFFSET), (CROPPING_OFFSET,CROPPING_OFFSET), (0,0))
images_pad = np.pad(private_test, padding_tuple) # Padding space is filled with zero values

#keypoints = masks.nonzero() # Get coordinates of measured height values
#keypoints_val = masks[keypoints] # Get measured height values

images_crop = []

for i in range(2, X+CROPPING_OFFSET):
  for j in range(2, Y+CROPPING_OFFSET):
    image_crop = images_pad[i-CROPPING_OFFSET : i+CROPPING_OFFSET+1, j-CROPPING_OFFSET: j+CROPPING_OFFSET+1,:]
  #print(f"X:{x} Y:{y} | Crop X start:{x-CROPPING_OFFSET} end:{x+CROPPING_OFFSET+1} Y start:{y-CROPPING_OFFSET} end:{y+CROPPING_OFFSET+1}")

    images_crop.append([i,j,image_crop])

images_crop = pd.DataFrame(images_crop, columns=["x","y","image"])

##Create MemMap, Save Cropped Images in NumpyArray, and Predict Height Values

In [ ]:
#Create MemMap to enable step-by-step loading of cropped images into MemMap - otherwise not enough RAM
map_private_image_crops = np.memmap('map_private_image_crops.dat', dtype='int16', mode='w+', shape=(X*Y,5,5,C))

In [ ]:
#Step-by-step loading into MemMap
for i in range(X*Y):
  map_private_image_crops[i] = np.array(list(images_crop.image[i]))

In [ ]:
#Load MemMap into normal NPArray - only transfer was problematic for 4k * 4k picture
private_image_crops = map_private_image_crops

In [ ]:
#Get dimensions of NP Array for reshape - Z is X*Y, A and B are Crop Image size 5, C is amount of colour bands
Z, A, B, C = private_image_crops.shape
private_image_crops = private_image_crops.reshape(Z, -1)

In [4]:
#Now apply model to get predictions for entire picture, then reshape into X * Y * Prediction
xgb_model_prediction = XGBRegressor()
xgb_model_prediction.load_model("model_xgb_crop_v1.json") #use if json file is on local fs
#drive.mount('/content/drive') - #use if model on Google Drive
#xgb_model_prediction.load_model('/content/drive/My Drive/model_xgb_crop_v1.json')

Mounted at /content/drive


In [ ]:
#Predict height values for tets image
private_image_pred = xgb_model_prediction.predict(private_image_crops)

##Reshape Predictions into Correct Form (4096x4096x1) and Save Them in .npy file

In [ ]:
#Reshape preds from flattened NP Array to Array with dimensions 1 * X * Y
private_image_pred = private_image_pred.reshape(-1,4096,4096)

In [ ]:
#Outlier detection and Adjustment - some predictions < 0, which is impossible due to nature of measurements (height above ground) - Fix: Set negative values to 0
private_image_pred = private_image_pred.clip(min=0)

In [ ]:
#Save Predictions in .npy file for upload
np.save("private_test_image_predictions", private_image_pred)

In [ ]:
#Visualizations of the predictions are kept in the backup notebook, as the backup notebook was our working notebook for the XGB model.